In [ ]:
!pip install -U spacy
#first install the library that would help us use BERT in an easy to use interface
#https://github.com/UKPLab/sentence-transformers/tree/master/sentence_transformers
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
!pip install opendatasets
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import opendatasets as od
import pandas
  
od.download(
    "https://www.kaggle.com/datasets/hamzafarooq50/hotel-listings-and-reviews?select=hotelReviewsInDubai__en2019100120191005.csv")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Your Kaggle Key: 


Abort: ignored

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [ ]:
!python -m spacy download en_core_web_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-17 21:04:20.627975: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-17 21:04:20.628142: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-17 21:04:20.628164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [ ]:
!ls

HotelListInDubai__en2019100120191005.csv     sample_data
hotelReviewsInDubai__en2019100120191005.csv


#Basic NLP

In [ ]:
# Data Cleaning

import re

#sample review from the IMDB dataset.
review = "<b>A touching movie!!</b> It is full of emotions and wonderful acting.<br> I could have sat through it a second time."

cleaned_review = re.sub(re.compile('<.*?>'), '', review) #removing HTML tags
cleaned_review = re.sub('[^A-Za-z0-9]+', ' ', cleaned_review) #taking only words

print(cleaned_review)

A touching movie It is full of emotions and wonderful acting I could have sat through it a second time 


In [ ]:
#Lowercase

cleaned_review = cleaned_review.lower()

print(cleaned_review)

a touching movie it is full of emotions and wonderful acting i could have sat through it a second time 


In [ ]:
# Tokenization

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

tokens = nltk.word_tokenize(cleaned_review)

print(cleaned_review)
print(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


a touching movie it is full of emotions and wonderful acting i could have sat through it a second time 
['a', 'touching', 'movie', 'it', 'is', 'full', 'of', 'emotions', 'and', 'wonderful', 'acting', 'i', 'could', 'have', 'sat', 'through', 'it', 'a', 'second', 'time']


In [ ]:
# Stop words removal

nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

filtered_review = [word for word in tokens if word not in stop_words] # removing stop words

print(filtered_review)

['touching', 'movie', 'full', 'emotions', 'wonderful', 'acting', 'could', 'sat', 'second', 'time']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer


lemmatizer = WordNetLemmatizer()

lemm_review = [lemmatizer.lemmatize(word) for word in filtered_review]

print(lemm_review)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


['touching', 'movie', 'full', 'emotion', 'wonderful', 'acting', 'could', 'sat', 'second', 'time']


# Moving to Deep Learning Part

In [ ]:
import os
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy

In [ ]:
text = """Looking for a hotel in New York near Times Square with free breakfast and cheaper 
than $100 for 2nd June which is really kids friendly and has a swimming pool and I want to stay there for 8 days"""
doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.render(doc, jupyter = True, style="ent")

In [ ]:
text = """Close to the Effiel Tower and is very high end with great shopping nearby"""
doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.render(doc, jupyter = True, style="ent")

In [ ]:
text = "I want to stay in a European city that filmed Game of Thrones and has very cheap booze and art galleries for 4 days"

In [ ]:
#text = """My very photogenic mother died in a freak accident (picnic, lightning) when I was three, and, save for a pocket of warmth in the darkest past, nothing of her subsists within the hollows and dells of memory, over which, if you can still stand my style (I am writing under observation), the sun of my infancy had set: surely, you all know those redolent remnants of day suspended, with the midges, about some hedge in bloom or suddenly entered and traversed by the rambler, at the bottom of a hill, in the summer dusk; a furry warmth, golden midges"""
doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.render(doc, jupyter = True, style="ent")

In [ ]:
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'

In [ ]:
import pandas as pd

from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl

embedder = SentenceTransformer('all-MiniLM-L6-v2')
#embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
import opendatasets as od
import pandas
  
od.download(
    "https://www.kaggle.com/datasets/hamzafarooq50/hotel-listings-and-reviews?resource=download&select=HotelListInDubai__en2019100120191005.csv")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: amlendukumawat
Your Kaggle Key: ··········


100%|██████████| 8.54M/8.54M [00:00<00:00, 79.8MB/s]

In [ ]:

import pandas as pds
  
# reading the XLSX file
file =('/content/hotel-listings-and-reviews/HotelListInDubai__en2019100120191005.csv')
df_list = pds.read_csv(file)
  
# displaying the contents of the XLSX file
df_list.head()

,Unnamed: 0,hotel_name,url,locality,reviews,tripadvisor_rating,checkIn,checkOut,price_per_night,booking_provider,no_of_deals,hotel_features
0,0,Four Points By Sheraton Downtown Dubai,http://www.tripadvisor.com/Hotel_Review-g29542...,Dubai,2046,NaN,2019/10/01,2019/10/05,$74,FourPoints.com,15,NaN
1,1,FIVE Palm Jumeirah Dubai,http://www.tripadvisor.com/Hotel_Review-g29542...,Dubai,5388,NaN,2019/10/01,2019/10/05,NaN,Booking.com,15,NaN
2,2,"Atlantis, The Palm",http://www.tripadvisor.com/Hotel_Review-g29542...,Dubai,25417,NaN,2019/10/01,2019/10/05,NaN,Booking.com,10,NaN
3,3,Citymax Hotel Bur Dubai,http://www.tripadvisor.com/Hotel_Review-g29542...,Dubai,3704,NaN,2019/10/01,2019/10/05,NaN,TripAdvisor,14,NaN
4,4,Premier Inn Dubai International Airport Hotel,http://www.tripadvisor.com/Hotel_Review-g29542...,Dubai,5215,NaN,2019/10/01,2019/10/05,NaN,Booking.com,14,NaN


In [ ]:

import pandas as pds
  
# reading the XLSX file
file =('/content/hotel-listings-and-reviews/hotelReviewsInDubai__en2019100120191005.csv')
df_reviews = pds.read_csv(file)
  
# displaying the contents of the XLSX file
df_reviews.head()

,Unnamed: 0,review_body,review_date,hotelName,hotelUrl
0,0,Just to say this is really an excellent hotel ...,"July 14, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...
1,1,"Found this pub by chance, what a great place, ...","July 12, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...
2,2,"House keeping is perfect , the rooms are alway...","July 9, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...
3,3,Although we had a few issues in terms of check...,"July 6, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...
4,4,I was stayed over 3 night in room ( 730 ) my f...,"July 4, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...


In [ ]:
df_reviews['hotelName'].value_counts()

0    Four Points By Sheraton Downtown Dubai\nName: hotel_name, dtype: object           54
8    Orient Guest House\nName: hotel_name, dtype: object                               54
23    Signature 1 Hotel Tecom\nName: hotel_name, dtype: object                         54
20    Golden Tulip Al Barsha\nName: hotel_name, dtype: object                          54
18    Winchester Grand Hotel Apartments\nName: hotel_name, dtype: object               54
1    FIVE Palm Jumeirah Dubai\nName: hotel_name, dtype: object                         54
9    Barjeel Heritage Guest House\nName: hotel_name, dtype: object                     54
29    London Creek Hotel Apartments\nName: hotel_name, dtype: object                   54
7    Address Dubai Marina\nName: hotel_name, dtype: object                             54
5    JW Marriott Hotel Dubai\nName: hotel_name, dtype: object                          54
2    Atlantis, The Palm\nName: hotel_name, dtype: object                               54
3    Citym

In [ ]:
df_reviews[['Hotel_Name_Clean','Extra']] = df_reviews.hotelName.str.split("\n",expand=True)
df_reviews['Hotel_Name_Clean'] = df_reviews['Hotel_Name_Clean'].str.slice(4,).str.strip()

In [ ]:
df_reviews['Hotel_Name_Clean'].drop_duplicates()

0             Four Points By Sheraton Downtown Dubai
54                          FIVE Palm Jumeirah Dubai
108                               Atlantis, The Palm
162                          Citymax Hotel Bur Dubai
216    Premier Inn Dubai International Airport Hotel
270                          JW Marriott Hotel Dubai
324                Four Points by Sheraton Bur Dubai
378                             Address Dubai Marina
432                               Orient Guest House
486                     Barjeel Heritage Guest House
540       DAMAC Towers by Paramount Hotels & Resorts
545                                 Hotel Beit Bahar
546                             Roda Boutique Villas
566                              Vida Emirates Hills
569                                   Vasantam Hotel
575                  Hyatt Place Dubai/Wasl District
612                                    BackPacker 16
634                    Crowne Plaza Dubai Apartments
635                Winchester Grand Hotel Apar

In [ ]:
df_reviews.head()

,Unnamed: 0,review_body,review_date,hotelName,hotelUrl,Hotel_Name_Clean,Extra
0,0,Just to say this is really an excellent hotel ...,"July 14, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...,Four Points By Sheraton Downtown Dubai,"Name: hotel_name, dtype: object"
1,1,"Found this pub by chance, what a great place, ...","July 12, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...,Four Points By Sheraton Downtown Dubai,"Name: hotel_name, dtype: object"
2,2,"House keeping is perfect , the rooms are alway...","July 9, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...,Four Points By Sheraton Downtown Dubai,"Name: hotel_name, dtype: object"
3,3,Although we had a few issues in terms of check...,"July 6, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...,Four Points By Sheraton Downtown Dubai,"Name: hotel_name, dtype: object"
4,4,I was stayed over 3 night in room ( 730 ) my f...,"July 4, 2019",0 Four Points By Sheraton Downtown Dubai\nN...,http://www.tripadvisor.com/Hotel_Review-g29542...,Four Points By Sheraton Downtown Dubai,"Name: hotel_name, dtype: object"


In [ ]:
df_combined = df_reviews.sort_values(['Hotel_Name_Clean']).groupby('Hotel_Name_Clean', sort=False).review_body.apply(''.join).reset_index(name='all_review')
  

In [ ]:
df_combined.head()

,Hotel_Name_Clean,all_review
0,Address Dubai Marina,"Excellent Hotel and service, i enjoyed my stay..."
1,Al SEEF Hotel,AMAZING palace with beautiful design the servi...
2,"Atlantis, The Palm",Nice hotel for the family. Everywhere in the h...
3,BackPacker 16,It's not a fancy hotel and it's not a real hos...
4,Barjeel Heritage Guest House,Only had two days here to break the long trip ...


In [ ]:
import re

df_combined['all_review'] = df_combined['all_review'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['all_review']= df_combined['all_review'].apply(lambda x: lower_case(x))


In [ ]:
df = df_combined

In [ ]:
df_sentences = df_combined.set_index("all_review")
df_sentences = df_sentences["Hotel_Name_Clean"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

28

In [ ]:
# df_sentences.head()

In [ ]:
list(df_sentences.keys())[:5]

['excellent hotel and service i enjoyed my stay and the business meetings i recommend this hotel to all my friends and anyone wants to stay in dubai all facilities in the room were very good and the internet also was very goodumair was vary good gay very friendly we just visited shades tottaly enjoyed the feel of the place food ambiance sarvice were upto the mark specially umair made us to like the place even more view and pool sid sitting area were classy a must visit umair was told me we start pool party the name is aquaholic we wating for that we come back very soon thankshave not stayed at the hotel but have used the restaurants the chefs are amazing a recent experience organizing an iftar get together for my team has left me feeling very satisfied and wanting to recommend address marina to others our booking was very last minute and was handled very efficiently by neha gidwani kohli lifestyle events executive not only did she handle our booking efficiently and patiently we kept ch

In [ ]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [ ]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

100%|██████████| 28/28 [00:00<00:00, 137357.32it/s]


In [ ]:
# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
corpus_embeddings[0].shape

(384,)

In [ ]:
corpus_embeddings[0]

array([-3.44585348e-03,  8.86060181e-04,  8.68610397e-04,  3.60185578e-02,
       -6.95023611e-02,  2.47212537e-02,  9.77890566e-02, -7.92217702e-02,
        1.00702122e-02, -7.40536749e-02,  1.12373689e-02, -4.12639827e-02,
        3.06909606e-02,  4.42646779e-02,  4.96442840e-02, -3.43752317e-02,
        7.74144605e-02, -6.06893785e-02, -7.87047595e-02, -7.74550671e-03,
       -4.96977791e-02, -1.50003433e-02, -3.74407060e-02, -5.43095432e-02,
       -1.11573189e-01,  1.14605315e-02, -1.74497385e-02,  5.20471595e-02,
       -3.99421988e-04, -5.95689155e-02,  1.95702389e-02,  1.44654170e-01,
        1.78821012e-02, -3.28862518e-02,  3.80284525e-02,  1.19153351e-01,
       -6.34656399e-02, -1.23817235e-01,  1.60905756e-02, -2.17618030e-02,
       -5.15343971e-04, -1.93868745e-02,  6.99806586e-02, -3.44565324e-02,
        4.28743614e-03,  6.81685144e-03, -4.64240182e-03,  1.37275066e-02,
        3.49233486e-02, -1.31877400e-02, -8.93387794e-02,  2.07583848e-02,
        2.24333517e-02, -

In [ ]:
# model = SentenceTransformer('all-MiniLM-L6-v2')
# paraphrases = util.paraphrase_mining(model, corpus)
#query_embeddings_p =  util.paraphrase_mining(model, queries,show_progress_bar=True)

In [ ]:
# import pickle as pkl
# with open("/content/drive/MyDrive/BertSentenceSimilarity/Pickles/corpus_embeddings.pkl" , "wb") as file_:
#  pkl.dump(corpus_embeddings,file_)

In [ ]:
queries = ['Hotel close to catacombs and has excellent breakfast',
           'Hotel which is close to the Louvre and is very high end with great shopping nearby'
           ]
query_embeddings = embedder.encode(queries,show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
query_embeddings.shape

(2, 384)

In [ ]:

import torch

# Query sentences:
queries = ['hotel that is close to the airport ',
           'Hotel with easy access for taxi'
           ]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("(Score: {:.4f})".format(score))
        print(corpus[idx], "(Score: {:.4f})".format(score))
        row_dict = df.loc[df['all_review']== corpus[idx]]
        print("paper_id:  " , row_dict['Hotel_Name_Clean'] , "\n")
    # for idx, distance in results[0:closest_n]:
    #     print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
    #     print("Paragraph:   ", corpus[idx].strip(), "\n" )
    #     row_dict = df.loc[df['all_review']== corpus[idx]]
    #     print("paper_id:  " , row_dict['Hotel'] , "\n")
    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """






Query: hotel that is close to the airport 

Top 5 most similar sentences in corpus:
(Score: 0.5451)
hotel is good and clean only food not good enough and expensive and we was not able to watch afcon on tv the have to go with time really i was happy with the hotel service price of the food is expensive and not good test the workers are very friendly nice and near easy transportation from and to airportfair price friendly stuff free bus to airport totally it was good experiment to me and my family also near most of tourism places and activities in dabai we booked this hotel for our layover and were pleasantly surprised it had free shuttle bus service to the hotel and to a number of attractions checkin and checkout were speedy and the staff very friendly and helpful it is very nice  cool and calmplace to stay and have a nice staff and good rates  rooms were clean and have all facilities  also parking is perfect have easy access to rooms and i would like to stay in future again and its

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
embeddings = model.encode(corpus)
#print(embeddings)

In [ ]:
query_embedding.shape

torch.Size([384])

In [ ]:
# Query sentences:
queries = ['Champs Élysées',
           'outside Gare du Nord'
           ]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("(Score: {:.4f})".format(score))
        print(corpus[idx], "(Score: {:.4f})".format(score))
        row_dict = df.loc[df['all_review']== corpus[idx]]
        print("paper_id:  " , row_dict['Hotel'] , "\n")





Query: Champs Élysées

Top 5 most similar sentences in corpus:
(Score: 0.3019)
great location  helpful friendly staff centrally located off avenue champs elysees and close to larc de triumphe if you want to dine out there are many restaurants in the neighbourhood the rooms are modern and cleanwe really enjoyed our stay the room with the balcony room arc de triomphe is really cool looking over the roofs towards arc and tour eiffel staff is very friendly and helpful even with topics outside the hotel we had some difficulties with public parking and are not french speaking and the location is perfect 5 minutes walk from metro arc and champs elysee we will definitely stay here again when we come back to paristruly have nothing but praise for this boutique hotel we had such a truly fantastic holiday here for a very short three nights could have easily stayed the entire week wish we had the hotel is quite stylish yet classy the absolute best part was the locationzone we walked everywhere

In [ ]:
df

,Hotel,all_review
0,25hours Hotel Terminus Nord,weve spent lots of time in paris and this was ...
1,Acacias Etoile Hotel,the hotel is great for value the breakfast sel...
2,COQ Hotel Paris,stayed for a short city break the hotel is a ...
3,Campanile Paris 14 - Maine Montparnasse,room was very clean transportation is very ne...
4,Cler Hotel,we had the best stay at cler hotel the locati...
...,...,...
89,Sofitel Paris Le Faubourg,4 years ago i was the last time at sofitel le ...
90,St Christopher's Gare du Nord Paris,when arriving to the area it felt a little dan...
91,St Christopher's Inn Canal Paris,ive stayed at st christopher inn canal in pari...
92,Touring Hotel,hotel is in a great location minutes walk fro...


In [ ]:
hits = util.semantic_search(query_embedding, embeddings, top_k=5)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
  print (hit)
  print("(Score: {:.4f})".format(hit['score']))
  print(corpus[hit['corpus_id']])
  row_dict = df.loc[df['all_review']== corpus[hit['corpus_id']]]
  print("paper_id:  " , row_dict['Hotel'] , "\n")

{'corpus_id': 77, 'score': 0.24323494732379913}
(Score: 0.2432)
the hotel is on a small beautiful unknown square right next to the cnam on the right bank close to grand boulevard st martin areas and super accessible from the gare du nord or montparnasse via the line 4   ive stayed here several times and always had a really nice view of the square   the staff is very welcoming and competentthe little palace hotel as its name suggests is small the rooms are quite tiny and there isnt much if any closet space but it wonderfully located and a beautiful spot if youre looking for a grand hotel this isnt for you but if you are looking for a nice little room to relax in when youre not walking the city i highly recommend this hotel i dont think it would be large enough for two people but for one it is enough room the staff was very helpful and kind and its minutes from wonderful shops and cafes i will definitely come back my personal favorite was the small terrace where i took my coffee in the m